# Loading data

In [ ]:
import osyris
import numpy as np

## Loading a Ramses output

We load a data output from a star formation simulation

In [ ]:
data = osyris.load(71, scale="au", path="osyrisdata")

and show a 2D histogram of the gas density versus the magnetic field strength

In [ ]:
osyris.plot.histogram(data["density"], data["B_field"],
                      norm="log", loglog=True)

## Selective loading

It is possible to load only a subset of the cells, by using custom functions to perform the selection.

As an example, to load all the cells with $\rho > 10^{-13}~{\rm g~cm}^{-3}$, we use a selection criterion

In [ ]:
data2 = osyris.load(71, scale="au", path="osyrisdata",
                    select={"density": lambda x : x > 1.0e-13})

and we can now see in the resulting histogram that all the low-density cells have been left out:

In [ ]:
osyris.plot.histogram(data2["density"], data2["B_field"],
                      norm="log", loglog=True)

Multiple selection criteria are ANDed:

In [ ]:
data3 = osyris.load(71, scale="au", path="osyrisdata",
                    select={"density": lambda x : x > 1.0e-13,
                            "xyz_x": lambda x : abs(x - 5517.) < 50})

In [ ]:
ind = np.argmax(data3["density"])
center = data3["xyz"][ind.values]
osyris.plane({"data": data3["density"], "norm": "log"},
             dx=200,
             origin=center,
             direction='z')